In [1]:
pip install neo4j pandas torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Note: you may need to restart the kernel to use updated packages.


In [2]:
from neo4j_connections import Neo4jConnection
import yaml
import pandas as pd 
import numpy as np

cuda


In [3]:
class GraphFetcher(object):
    def __init__(self, configuration):
        self.configuration = configuration 

    def load_config(self):
        global config
        with open(self.configuration, 'r') as config_file:
            config = yaml.load(config_file, Loader = yaml.FullLoader)

In [4]:
configure = 'config.yaml'
graphFetcher = GraphFetcher(configure)
graphFetcher.load_config()
connection = Neo4jConnection(config)

In [5]:
patient_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/patient.csv')
diagnosis_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/diagnosis.csv')
admission_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/admissionDx.csv')
apachepatientresult_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/apachePatientResult.csv')
pasthistory_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/pastHistory.csv')
treatment_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/treatment.csv')

In [ ]:
vitalPeriodic_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/vitalPeriodic.csv')

In [6]:
apacheapsvar_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/apacheApsVar.csv')

In [34]:
patient_df['uniquepid'].nunique()

139367

Patient 

In [41]:
#Get distinct values of a dataframe based on a column
p_unique_df = patient_df.drop_duplicates(subset=["uniquepid"])
p_unique_df

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141178,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
4,141196,128943,Male,71,Caucasian,67,109,NaN,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665
6,141203,128948,Female,77,Caucasian,66,90,"Arrest, respiratory (without cardiac arrest)",160.0,22:23:00,...,Floor,1,admit,70.2,70.3,03:48:00,1869,Floor,Alive,002-23234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200852,3353216,2743066,Female,50,African American,458,1107,"Cystectomy, other reasons",165.1,19:19:00,...,Operating Room,1,admit,55.4,61.8,22:49:00,3090,Home,Alive,035-2734
200854,3353235,2743084,Male,50,Caucasian,458,1109,"CHF, congestive heart failure",175.3,04:55:00,...,Emergency Department,1,admit,90.0,99.2,23:18:00,1069,Telemetry,Alive,035-16382
200855,3353237,2743086,Female,79,Caucasian,458,1106,"Embolus, pulmonary",162.6,01:45:00,...,Direct Admit,1,admit,78.4,81.4,23:08:00,1269,Step-Down Unit (SDU),Alive,035-751
200857,3353254,2743102,Male,81,Caucasian,459,1108,"Bleeding, lower GI",185.4,07:43:00,...,Emergency Department,1,admit,83.9,92.9,19:25:00,431,Step-Down Unit (SDU),Alive,035-19511


In [46]:
unique_patients = p_unique_df[['gender', 'age', 'ethnicity', 'uniquepid']]
unique_patients

,gender,age,ethnicity,uniquepid
0,Female,70,Caucasian,002-34851
1,Female,52,Caucasian,002-33870
3,Male,68,Caucasian,002-5276
4,Male,71,Caucasian,002-37665
6,Female,77,Caucasian,002-23234
...,...,...,...,...
200852,Female,50,African American,035-2734
200854,Male,50,Caucasian,035-16382
200855,Female,79,Caucasian,035-751
200857,Male,81,Caucasian,035-19511


In [22]:
diagnosis_df.columns

Index(['diagnosisid', 'patientunitstayid', 'activeupondischarge',
       'diagnosisoffset', 'diagnosisstring', 'icd9code', 'diagnosispriority'],
      dtype='object')

In [7]:
patient_df.loc[patient_df['unitvisitnumber'] > 1]

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
2,141179,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
4,141196,128943,Male,71,Caucasian,67,109,NaN,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665
13,141263,128998,Male,19,Caucasian,59,91,NaN,188.0,02:24:29,...,ICU to SDU,2,stepdown/other,NaN,NaN,16:57:00,475,Home,Alive,002-47110
20,141289,129020,Female,61,Caucasian,67,109,Pneumothorax,162.6,17:32:00,...,Floor,2,readmit,NaN,NaN,21:28:00,1283,Other Hospital,Alive,002-67735
22,141297,129026,Male,63,Caucasian,73,85,"Sepsis, pulmonary",162.6,04:18:00,...,Floor,2,readmit,NaN,74.4,03:41:00,1869,Death,Expired,002-30269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200845,3353196,2743055,Female,66,Caucasian,458,1104,Atelectasis,157.5,06:38:00,...,Step-Down Unit (SDU),3,readmit,71.5,93.7,23:10:00,1231,Step-Down Unit (SDU),Alive,035-18808
200847,3353198,2743055,Female,66,Caucasian,458,1106,Coma/change in level of consciousness (for hep...,157.5,06:38:00,...,Step-Down Unit (SDU),5,readmit,71.5,85.6,20:51:00,23031,Step-Down Unit (SDU),Alive,035-18808
200848,3353199,2743055,Female,66,Caucasian,458,1104,"Pneumonia, aspiration",157.5,06:38:00,...,Other ICU,2,transfer,71.5,96.8,23:37:00,10547,Step-Down Unit (SDU),Alive,035-18808
200849,3353200,2743055,Female,66,Caucasian,458,1109,"Hypovolemia (including dehydration, Do not inc...",157.5,06:38:00,...,Floor,6,readmit,71.5,60.7,18:12:00,6483,Step-Down Unit (SDU),Alive,035-18808


In [109]:
patient_df['patienthealthsystemstayid'].value_counts()

590180     17
498628     13
2416711    12
631956     11
603432     11
           ..
1096609     1
1096598     1
1096592     1
1096590     1
2743110     1
Name: patienthealthsystemstayid, Length: 166355, dtype: int64

In [121]:
patient_df['uniquepid'].nunique()

139367

In [48]:
def create_unique_patients_neo4j(node_list):
    pa_node_query = """
    UNWIND $node_list as node
    CREATE( p: patient {
        gender: node.gender,
        age: toInteger(node.age),
        ethnicity: node.ethnicity,
        uniquepid: node.uniquepid
    })
    """
    
    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(pa_node_query, {"node_list": records})
    

In [49]:
create_unique_patients_neo4j(unique_patients)

In [51]:
def create_patientstay_nodes_neo4j(node_list):
    patientstay_node_query = """
    UNWIND $node_list as node
    CREATE( pus: patientunitstay {
        patientunitstayid: toInteger(node.patientunitstayid),
        patienthealthsystemstayid: toInteger(node.patienthealthsystemstayid),
        hospitalid: toInteger(node.hospitalid),
        wardid: toInteger(node.wardid),
        apacheadmissiondx: node.apacheadmissiondx,
        admissionheight: toFloat(node.admissionheight),
        hospitaladmittime24: node.hospitaladmittime24,
        hospitaladmitoffset: toInteger(node.hospitaladmitoffset),
        hospitaladmitsource: node.hospitaladmitsource,
        hospitaldischargeyear: toInteger(node.hospitaldischargeyear),
        hospitaldischargetime24: node.hospitaldischargetime24,
        hospitaldischargeoffset: toInteger(node.hospitaldischargeoffset),
        hospitaldischargelocation: node.hospitaldischargelocation,
        hospitaldischargestatus: node.hospitaldischargestatus,
        unittype: node.unittype,
        unitadmittime24: node.unitadmittime24,
        unitadmitsource: node.unitadmitsource,
        unitvisitnumber: toInteger(node.unitvisitnumber),
        unitstaytype: node.unitstaytype,
        admissionweight: toFloat(node.admissionweight),
        dischargeweight: toFloat(node.dischargeweight),
        unitdischargetime24: node.unitdischargetime24,
        unitdischargeoffset: toInteger(node.unitdischargeoffset),
        unitdischargelocation: node.unitdischargelocation,
        unitdischargestatus: node.unitdischargestatus,
        uniquepid: node.uniquepid
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(patientstay_node_query, {"node_list": records})

In [52]:
create_patientstay_nodes_neo4j(patient_df)

In [56]:
diagnosis_df.columns

Index(['diagnosisid', 'patientunitstayid', 'activeupondischarge',
       'diagnosisoffset', 'diagnosisstring', 'icd9code', 'diagnosispriority'],
      dtype='object')

In [57]:
diagnosis_df

,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
0,4222318,141168,False,72,cardiovascular|chest pain / ASHD|coronary arte...,"414.00, I25.10",Other
1,3370568,141168,True,118,cardiovascular|ventricular disorders|cardiomyo...,NaN,Other
2,4160941,141168,False,72,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
3,4103261,141168,True,118,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
4,3545241,141168,True,118,cardiovascular|ventricular disorders|congestiv...,"428.0, I50.9",Other
...,...,...,...,...,...,...,...
2710667,46330138,3353251,False,11304,renal|disorder of urinary tract / renal system...,"599.0, N39.0",Other
2710668,46150971,3353251,False,4080,cardiovascular|shock / hypotension|sepsis,"038.9, A41.9",Major
2710669,46259796,3353254,True,41,renal|disorder of kidney|acute renal failure|d...,"584.9, N17.9",Major
2710670,46204273,3353254,True,41,gastrointestinal|GI bleeding / PUD|lower GI bl...,"578.9, K92.2",Primary


In [58]:
diagnosis_df

,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
0,4222318,141168,False,72,cardiovascular|chest pain / ASHD|coronary arte...,"414.00, I25.10",Other
1,3370568,141168,True,118,cardiovascular|ventricular disorders|cardiomyo...,NaN,Other
2,4160941,141168,False,72,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
3,4103261,141168,True,118,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
4,3545241,141168,True,118,cardiovascular|ventricular disorders|congestiv...,"428.0, I50.9",Other
...,...,...,...,...,...,...,...
2710667,46330138,3353251,False,11304,renal|disorder of urinary tract / renal system...,"599.0, N39.0",Other
2710668,46150971,3353251,False,4080,cardiovascular|shock / hypotension|sepsis,"038.9, A41.9",Major
2710669,46259796,3353254,True,41,renal|disorder of kidney|acute renal failure|d...,"584.9, N17.9",Major
2710670,46204273,3353254,True,41,gastrointestinal|GI bleeding / PUD|lower GI bl...,"578.9, K92.2",Primary


In [75]:
diagnosis_df.iloc[0].diagnosisstring

'cardiovascular|chest pain / ASHD|coronary artery disease|known'

In [18]:
def create_diagnosis_nodes_neo4j(node_list):
    diagnosis_node_query = """
    UNWIND $node_list as node
    CREATE( d: diagnosis {
        diagnosisid: toInteger(node.diagnosisid),
        patientunitstayid: toInteger(node.patientunitstayid),
        activeupondischarge: node.activeupondischarge,
        diagnosisoffset: toInteger(node.diagnosisoffset),
        diagnosisstring: node.diagnosisstring,
        icd9code: node.icd9code,
        diagnosisPriority: node.diagnosisPriority
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(diagnosis_node_query, {"node_list": records})

In [19]:
create_diagnosis_nodes_neo4j(diagnosis_df)

In [78]:
admission_df

,admissiondxid,patientunitstayid,admitdxenteredoffset,admitdxpath,admitdxname,admitdxtext
0,8023778,2900217,62,admission diagnosis|Operative Organ Systems|Or...,Cardiovascular,Cardiovascular
1,8023777,2900217,62,admission diagnosis|Was the patient admitted f...,Yes,Yes
2,8023779,2900217,62,admission diagnosis|All Diagnosis|Operative|Di...,Aortic and Mitral valve replacement,Aortic and Mitral valve replacement
3,7929318,2900240,53,admission diagnosis|Non-operative Organ System...,Gastrointestinal,Gastrointestinal
4,7929317,2900240,53,admission diagnosis|Was the patient admitted f...,No,No
...,...,...,...,...,...,...
626853,7902610,2900186,55,admission diagnosis|Non-operative Organ System...,Cardiovascular,Cardiovascular
626854,8041507,2900196,48,admission diagnosis|Non-operative Organ System...,Metabolic/Endocrine,Metabolic/Endocrine
626855,8041506,2900196,48,admission diagnosis|Was the patient admitted f...,No,No
626856,8041508,2900196,48,admission diagnosis|All Diagnosis|Non-operativ...,Hypoglycemia,Hypoglycemia


In [83]:
apachepatientresult_df.columns

Index(['apachepatientresultsid', 'patientunitstayid', 'physicianspeciality',
       'physicianinterventioncategory', 'acutephysiologyscore', 'apachescore',
       'apacheversion', 'predictedicumortality', 'actualicumortality',
       'predictediculos', 'actualiculos', 'predictedhospitalmortality',
       'actualhospitalmortality', 'predictedhospitallos', 'actualhospitallos',
       'preopmi', 'preopcardiaccath', 'ptcawithin24h', 'unabridgedunitlos',
       'unabridgedhosplos', 'actualventdays', 'predventdays',
       'unabridgedactualventdays'],
      dtype='object')

In [108]:
apachepatientresult_df['actualiculos']

0          2.4972
1          2.4972
2          3.3423
3          3.3423
4          1.2979
           ...   
297059    11.2909
297060     0.2993
297061     0.2993
297062     0.6444
297063     0.6444
Name: actualiculos, Length: 297064, dtype: float64

In [53]:
def create_apachepatientresult_nodes(node_list):
    apr_node_query = """
    UNWIND $node_list as node
    CREATE(apr: apachepatientresult {
        apachepatientresultsid: toInteger(node.apachepatientresultsid),
        patientunitstayid: toInteger(node.patientunitstayid),
        physicianspeciality: node.physicianspeciality,
        physicianinterventioncategory: node.physicianinterventioncategory,
        acutephysiologyscore: toInteger(node.acutephysiologyscore),
        apachescore: toInteger(node.apachescore),
        apacheversion: node.apacheversion,
        predictedicumortality: node.predictedicumortality,
        actualicumortality: node.actualicumortality,
        predictediculos: toFloat(node.predictediculos),
        actualiculos: toFloat(node.actualiculos),
        predictedhospitalmortality: node.predictedhospitalmortality,
        actualhospitalmortality: node.actualhospitalmortality,
        predictedhospitallos: toFloat(node.predictedhospitallos),
        actualhospitallos: toFloat(node.actualhospitallos),
        preopmi: toInteger(node.preopmi),
        preopcardiaccath: toInteger(node.preopcardiaccath),
        ptcawithin24h: toInteger(node.ptcawithin24h),
        unabridgedunitlos: toFloat(node.unabridgedunitlos),
        unabridgedhosplos: toFloat(node.unabridgedhosplos),
        actualventdays: toFloat(node.actualventdays),
        predventdays: toFloat(node.predventdays),
        unabridgedactualventdays: toFloat(node.unabridgedactualventdays)
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(apr_node_query, {"node_list": records})

In [54]:
create_apachepatientresult_nodes(apachepatientresult_df)

In [90]:
pasthistory_df.columns

Index(['pasthistoryid', 'patientunitstayid', 'pasthistoryoffset',
       'pasthistoryenteredoffset', 'pasthistorynotetype', 'pasthistorypath',
       'pasthistoryvalue', 'pasthistoryvaluetext'],
      dtype='object')

In [20]:
def create_pasthistory_nodes(node_list):
    pasthistory_query = """
    UNWIND $node_list as node
    CREATE(ph: pasthistory {
        pasthistoryid: toInteger(node.pasthistoryid),
        patientunitstayid: toInteger(node.patientunitstayid),
        pasthistoryoffset: toInteger(node.pasthistoryoffset),
        pasthistoryenteredoffset: toInteger(node.pasthistoryenteredoffset),
        pasthistorynotetype: node.pasthistorynotetype,
        pasthistorypath: node.pasthistorypath,
        pasthistoryvalue: node.pasthistoryvalue,
        pasthistoryvaluetext: node.pasthistoryvaluetext
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(pasthistory_query, {"node_list": records})

In [21]:
create_pasthistory_nodes(pasthistory_df)

In [22]:
admission_df

,admissiondxid,patientunitstayid,admitdxenteredoffset,admitdxpath,admitdxname,admitdxtext
0,8023778,2900217,62,admission diagnosis|Operative Organ Systems|Or...,Cardiovascular,Cardiovascular
1,8023777,2900217,62,admission diagnosis|Was the patient admitted f...,Yes,Yes
2,8023779,2900217,62,admission diagnosis|All Diagnosis|Operative|Di...,Aortic and Mitral valve replacement,Aortic and Mitral valve replacement
3,7929318,2900240,53,admission diagnosis|Non-operative Organ System...,Gastrointestinal,Gastrointestinal
4,7929317,2900240,53,admission diagnosis|Was the patient admitted f...,No,No
...,...,...,...,...,...,...
626853,7902610,2900186,55,admission diagnosis|Non-operative Organ System...,Cardiovascular,Cardiovascular
626854,8041507,2900196,48,admission diagnosis|Non-operative Organ System...,Metabolic/Endocrine,Metabolic/Endocrine
626855,8041506,2900196,48,admission diagnosis|Was the patient admitted f...,No,No
626856,8041508,2900196,48,admission diagnosis|All Diagnosis|Non-operativ...,Hypoglycemia,Hypoglycemia


In [23]:
def create_admission_nodes(node_list):
    admission_query = """
    UNWIND $node_list as node
    CREATE(ad: admissiondx {
        admissiondxid: toInteger(node.admissiondxid),
        patientunitstayid: toInteger(node.patientunitstayid),
        admitdxenteredoffset: toInteger(node.admitdxenteredoffset),
        admitdxpath: node.admitdxpath,
        admitdxname: node.admitdxname,
        admitdxtext: node.admitdxtext
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(admission_query, {"node_list": records})

In [24]:
create_admission_nodes(admission_df)

In [14]:
treatment_df

,treatmentid,patientunitstayid,treatmentoffset,treatmentstring,activeupondischarge
0,8399138,242040,198,cardiovascular|hypertension|angiotensin II rec...,False
1,8626134,242040,198,cardiovascular|myocardial ischemia / infarctio...,False
2,8517569,242040,198,infectious diseases|medications|therapeutic an...,False
3,9597686,242040,616,cardiovascular|non-operative procedures|diagno...,False
4,9334096,242040,618,infectious diseases|medications|therapeutic an...,True
...,...,...,...,...,...
3688740,82519717,3353251,291,pulmonary|radiologic procedures / bronchoscopy...,False
3688741,82608952,3353251,291,pulmonary|ventilation and oxygenation|mechanic...,False
3688742,82541591,3353254,41,cardiovascular|intravenous fluid|normal saline...,True
3688743,82230708,3353254,41,cardiovascular|intravenous fluid|blood product...,True


In [17]:
diagnosis_df['diagnosisstring'][:20]

0     cardiovascular|chest pain / ASHD|coronary arte...
1     cardiovascular|ventricular disorders|cardiomyo...
2               pulmonary|disorders of the airways|COPD
3               pulmonary|disorders of the airways|COPD
4     cardiovascular|ventricular disorders|congestiv...
5     cardiovascular|arrhythmias|atrial fibrillation...
6       renal|disorder of kidney|chronic kidney disease
7     cardiovascular|chest pain / ASHD|coronary arte...
8     cardiovascular|ventricular disorders|congestiv...
9     cardiovascular|ventricular disorders|cardiomyo...
10      renal|disorder of kidney|chronic kidney disease
11    cardiovascular|arrhythmias|atrial fibrillation...
12    infectious diseases|skin, bone and joint infec...
13    pulmonary|respiratory failure|acute respirator...
14       endocrine|glucose metabolism|diabetes mellitus
15     neurologic|altered mental status / pain|dementia
16    infectious diseases|skin, bone and joint infec...
17                         neurologic|seizures|s

In [8]:
apacheapsvar_df.columns

Index(['apacheapsvarid', 'patientunitstayid', 'intubated', 'vent', 'dialysis',
       'eyes', 'motor', 'verbal', 'meds', 'urine', 'wbc', 'temperature',
       'respiratoryrate', 'sodium', 'heartrate', 'meanbp', 'ph', 'hematocrit',
       'creatinine', 'albumin', 'pao2', 'pco2', 'bun', 'glucose', 'bilirubin',
       'fio2'],
      dtype='object')

In [11]:
def create_apacheapsvar_nodes(node_list):
    apacheapsvar_query = """
    UNWIND $node_list as node
    CREATE(aav: apacheapsvar {
        apacheapsvarid: toInteger(node.apacheapsvarid),
        patientunitstayid: toInteger(node.patientunitstayid),
        intubated: toInteger(node.intubated),
        vent: toInteger(node.vent),
        dialysis: toInteger(node.dialysis),
        eyes: toInteger(node.eyes),
        motor: toInteger(node.motor),
        verbal: toInteger(node.verbal),
        meds: toInteger(node.meds),
        urine: toFloat(node.urine),
        wbc: toFloat(node.wbc),
        temperature: toFloat(node.temperature),
        respiratoryrate: toFloat(node.respiratoryrate),
        sodium: toFloat(node.sodium),
        heartrate: toFloat(node.heartrate),
        meanbp: toFloat(node.meanbp),
        ph: toFloat(node.ph),
        hematocrit: toFloat(node.hematocrit),
        creatinine: toFloat(node.creatinine),
        albumin: toFloat(node.albumin),
        pao2: toFloat(node.pao2),
        pco2: toFloat(node.pco2),
        bun: toFloat(node.bun),
        glucose: toFloat(node.glucose),
        bilirubin: toFloat(node.bilirubin),
        fio2: toFloat(node.fio2)
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(apacheapsvar_query, {"node_list": records})

In [12]:
create_apacheapsvar_nodes(apacheapsvar_df)

In [6]:
def create_labs_nodes(node_list):
    labs_query = """
    UNWIND $node_list as node
    CREATE(l: lab {
        labid: toInteger(node.labid),
        patientunitstayid: toInteger(node.patientunitstayid),
        labresultoffset: toInteger(node.labresultoffset),
        labtypeid: toFloat(node.labtypeid),
        labname: node.labname,
        labresult: toFloat(node.labresult),
        labresulttext: node.labresulttext,
        labmeasurenamesystem: node.labmeasurenamesystem,
        labmeasurenameinterface: node.labmeasurenameinterface,
        labresultrevisedoffset: toInteger(node.labresultrevisedoffset)
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(labs_query, {"node_list": records})

In [20]:
chunksize = 10 ** 6
filename = '../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/lab.csv'
for chunk in pd.read_csv(filename, chunksize=chunksize):
    print(chunk['labname'].value_counts())
    
    #print(chunk['labname'].nunique())
    break

bedside glucose              94536
potassium                    41059
Hct                          33029
Hgb                          32663
sodium                       31009
                             ...  
T4                               3
Gentamicin - trough              2
Legionella pneumophila Ab        2
Gentamicin - peak                1
RPR titer                        1
Name: labname, Length: 123, dtype: int64


In [ ]:
filename = '../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/lab.csv'
lab_df = pd.read_csv(filename)
unique_labs = lab_df['labname'].unique()

In [4]:
print(type(unique_labs))

<class 'numpy.ndarray'>


In [12]:
unique_labs_df = pd.DataFrame(unique_labs, columns = ["labname"])
print(type(unique_labs_df))
print(unique_labs_df)

<class 'pandas.core.frame.DataFrame'>
                  labname
0              fibrinogen
1                PT - INR
2               magnesium
3                      PT
4                      pH
..                    ...
153       Amikacin - peak
154            Vent Other
155  HSV 1&2 IgG AB titer
156            HIV 1&2 AB
157        HSV 1&2 IgG AB

[158 rows x 1 columns]


In [17]:
def create_labname_nodes(node_list):
    labname_query = """
    UNWIND $node_list as node
    CREATE(l: lab {
        labname: node.labname
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 50

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(labname_query, {"node_list": records})

In [18]:
create_labname_nodes(unique_labs_df)

In [57]:
def create_lab_relationship(node_list):
    # lab_relation_query = """
    # UNWIND $node_list as node
    # CALL apoc.periodic.iterate(
    #   "MATCH (pus:patientunitstay)
    #    MATCH (l:lab) 
    #    WHERE l.labname = node.labname AND pus.patientunitstayid = node.patientunitstayid RETURN l,pus",
    #   "MERGE (pus)-[:HAS_LAB{patientunitstayid : toInteger(node.patientunitstayid),
    #     labresultoffset: toInteger(node.labresultoffset),
    #     labtypeid: toFloat(node.labtypeid),
    #     labresult: toFloat(node.labresult),
    #     labresulttext: node.labresulttext,
    #     labmeasurenamesystem: node.labmeasurenamesystem,
    #     labmeasurenameinterface: node.labmeasurenameinterface,
    #     labresultrevisedoffset: toInteger(node.labresultrevisedoffset)
    #   }]->(l)",
    #   {batchSize:10000, parallel:true})
    #   YIELD *
    # """

    lab_r_query = """
       UNWIND $node_list as node
       MATCH (pus:patientunitstay)
       MATCH (l:lab) 
       WHERE l.labname = node.labname AND pus.patientunitstayid = node.patientunitstayid
       WITH l, pus, node
       MERGE (pus)-[:HAS_LAB{patientunitstayid : toInteger(node.patientunitstayid),
        labresultoffset: toInteger(node.labresultoffset),
        labtypeid: toFloat(node.labtypeid),
        labresult: node.labresult,
        labresulttext: node.labresulttext,
        labresultrevisedoffset: toInteger(node.labresultrevisedoffset)
      }]->(l)
    """


    connection = Neo4jConnection(config)
    
    
    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(lab_r_query, {"node_list": records})

In [59]:
chunksize = 10 ** 6
filename = '../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/lab.csv'
for chunk in pd.read_csv(filename, chunksize=chunksize):
    chunk['labresult'].fillna("not available", inplace=True)
    chunk['labresulttext'].fillna("not available", inplace=True)
    #print(chunk['labresult'].isna().sum()) 
    
    create_lab_relationship(chunk)

In [ ]:
create_lab_relationship(lab_df)

In [10]:
def create_vitalPeriodic_nodes(node_list):
    vitalperiodic_query = """
    UNWIND $node_list as node
    CREATE(vp: vitalperiodic {
        vitalperiodicid: toInteger(node.vitalperiodicid),
        patientunitstayid: toInteger(node.patientunitstayid),
        observationoffset: toInteger(node.observationoffset),
        temperature: toInteger(node.temperature),
        sao2: toInteger(node.sao2),
        heartrate: toInteger(node.heartrate),
        respiration: toInteger(node.respiration),
        cvp: toInteger(node.cvp),
        etco2: toInteger(node.etco2),
        systemicsystolic: toInteger(node.systemicsystolic),
        systemicdiastolic: toInteger(node.systemicdiastolic),
        systemicmean: toInteger(node.systemicmean),
        pasystolic: toInteger(node.pasystolic),
        pamean: toInteger(node.pamean),
        st1: toFloat(node.st1),
        st2: toFloat(node.st2),
        st3: toFloat(node.st3),
        icp: toInteger(node.icp)
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 5000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(vitalperiodic_query, {"node_list": records})

In [11]:
chunksize = 10 ** 6
vpname = '../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/vitalPeriodic.csv'
for chunk in pd.read_csv(vpname, chunksize=chunksize):
    create_vitalPeriodic_nodes(chunk)
print("All VItalPeriodic nodes has been uploaded")

All VItalPeriodic nodes has been uploaded


In [15]:
vap_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/vitalAperiodic.csv')

In [19]:
def create_vitalAperiodic_nodes(node_list):
    vitalaperiodic_query = """
    UNWIND $node_list as node
    CREATE(vap: vitalaperiodic {
        vitalaperiodicid: toInteger(node.vitalaperiodicid),
        patientunitstayid: toInteger(node.patientunitstayid),
        observationoffset: toInteger(node.observationoffset),
        noninvasivesystolic: toFloat(node.noninvasivesystolic),
        noninvasivediastolic: toFloat(node.noninvasivediastolic),
        noninvasivemean: toFloat(node.noninvasivemean),
        paop: toFloat(node.paop),
        cardiacoutput: toFloat(node.cardiacoutput),
        cardiacinput: toFloat(node.cardiacinput),
        svr: toFloat(node.svr),
        svri: toFloat(node.svri),
        pvr: toFloat(node.pvr),
        pvri: toFloat(node.pvri)
    })
    """

    connection = Neo4jConnection(config)

    batch_len = 10000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(vitalaperiodic_query, {"node_list": records})

In [20]:
create_vitalAperiodic_nodes(vap_df)

In [23]:
respiratory_charting_df = pd.read_csv('../../../PyG-Neo4j/dataset/eicu-collaborative-research-database-2.0/respiratoryCharting.csv')
respiratory_charting_df

/home/nasim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue
0,3,221468,4679,4679,respFlowCareData,RT Vent On/Off,Continued
1,4,221468,4679,4679,respFlowSettings,FiO2,40
2,5,221468,4679,4679,respFlowSettings,Vent Rate,14
3,6,221468,4679,4679,respFlowSettings,Tidal Volume (set),450
4,7,221468,4679,4679,respFlowSettings,TV/kg IBW,5.4720
...,...,...,...,...,...,...,...
20168171,255567518,3335097,15149,15149,respFlowPtVentData,Peak Insp. Pressure,14
20168172,255567519,3335097,15149,15149,respFlowPtVentData,Exhaled MV,7.9
20168173,255567520,3335097,15149,15149,respFlowPtVentData,Mean Airway Pressure,6
20168174,255567527,3331083,126,126,respFlowSettings,FiO2,40


In [24]:
def respiratory_charting_nodes(node_list):
    respiratory_query = """
    UNWIND $node_list as node
    CREATE(rc: respiratorycharting {
        respchartid: toInteger(node.respchartid),
        patientunitstayid: toInteger(node.patientunitstayid),
        respchartoffset: toInteger(node.respchartoffset),
        respchartentryoffset: toInteger(node.respchartentryoffset),
        respcharttypecat: node.respcharttypecat,
        respchartvaluelabel: node.respchartvaluelabel,
        respchartvalue: node.respchartvalue
    })
    """
    
    connection = Neo4jConnection(config)

    batch_len = 10000

    for batch_start in range(0, len(node_list), batch_len):
        batch_end = batch_start + batch_len
        records = node_list.iloc[batch_start:batch_end].to_dict("records")
        connection.query(respiratory_query, {"node_list": records})

In [25]:
respiratory_charting_nodes(respiratory_charting_df)